## Import the necessary modules

In [10]:
import mysql.connector as connector
import os
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error
import logging
import datetime
from mysql.connector import FieldType

## Create a logger

In [11]:
logger = logging.getLogger("[Database Structures and Management with MySQL]")
logging.basicConfig(filename='management-with-mysql.log', encoding='utf-8', level=logging.DEBUG, format='%(asctime)s ==> %(message)s', datefmt='%m/%d/%Y %I:%M:%S')

## Establish a connection using MySQlConnectionPool 

In [12]:
dbconfig={"user":"root", "password":os.environ["MYSQL_PASSWORD"], "port":33061, "host":"localhost"}
try:
    logger.info("Creating connection pools")
    pool = MySQLConnectionPool(pool_size=4, pool_name="pool_b", **dbconfig)
    print(f"The connection pool is created with a name: {pool.pool_name}")
    logger.info(f"The connection pool is created with a name: {pool.pool_name}")
    print(f"The pool size is: {pool.pool_size}")
    logger.info(f"The pool size is: {pool.pool_size}")
except Error as err:
    print(f"Error code: {err.errno}")
    logger.info(f"Error code: {err.errno}")
    print(f"Error message: {err.msg}")
    logger.info(f"Error message: {err.msg}")

The connection pool is created with a name: pool_b
The pool size is: 4


## Creating a connection from the connection pool

In [13]:
first_connection = pool.get_connection()
print("First connections established between MySQL and Python")
logger.info("First connections established between MySQL and Python")

First connections established between MySQL and Python


## Create a cursor objects

In [14]:
print("Creating first cursor object from the first connection")
logger.info("Creating first cursor object from the first connection")
first_cursor = first_connection.cursor(buffered=True)
print("First Cursor object created to communicate with MySQL using Python.")
logger.info("First Cursor object created to communicate with MySQL using Python.")

Creating first cursor object from the first connection
First Cursor object created to communicate with MySQL using Python.


## Create Database 

In [15]:
# Get a cursor object from the cursor pool
database_name = "db_meta_dsm_mysql"
drop_database_query = f"""DROP DATABASE IF EXISTS {database_name}"""
first_cursor.execute(drop_database_query)
logger.info("Dropping Database if it already exists.")

create_database_query: str = f"""CREATE DATABASE IF NOT EXISTS {database_name}"""
print("Creating Database.")
logger.info("Creating Database.")
first_cursor.execute(create_database_query)
logger.info("Database created.")
print("Database created.")

Creating Database.
Database created.


In [16]:
# Check to see that the database was created
list_of_databases: list = []
first_cursor.execute("SHOW DATABASES;")
databases = first_cursor.fetchall()
for database in databases:
    db_name: str = database[0]
    list_of_databases.append(db_name)
    if db_name == database_name:
        print(f"Database '{database_name}' was successfully created")
        logger.info(f"Database '{database_name}' was successfully created.")
        break
print(list_of_databases)
logger.info(list_of_databases)

Database 'db_meta_dsm_mysql' was successfully created
['CVD', 'LittleLemonDB', 'PC', 'STAFF_LOCATIONS', 'db_Exercise', 'db_hr', 'db_learner', 'db_little_lemon', 'db_meta', 'db_meta_dsm_mysql']


In [17]:
# Set the new created database as the database to use
first_cursor.execute(f"USE {database_name}")
print(f"Database '{database_name}' is set for use.")
logger.info(f"Database '{database_name}' is set for use.")

Database 'db_meta_dsm_mysql' is set for use.


## Assert Database contains no Tables

In [18]:
## Expect an empty list to be returned 
first_cursor.execute("SHOW TABLES;")
results = first_cursor.rowcount
assert results == 0

## Create Table

In [19]:
create_table_query = """CREATE TABLE IF NOT EXISTS tbl_orders (OrderID INT NOT NULL PRIMARY KEY, ClientID VARCHAR(10),  ProductID VARCHAR(10),  Quantity INT, Cost DECIMAL(6,2));"""
first_cursor.execute(create_table_query)
print("Table tbl_orders successfully created")
logger.info("Table tbl_orders successfully created")

Table tbl_orders successfully created


## Populate the table with data

In [20]:
insert_into_tbl_order_query = """INSERT INTO tbl_orders (OrderID, ClientID, ProductID , Quantity, Cost) VALUES (1, "Cl1", "P1", 10, 500), (2, "Cl2", "P2", 5, 100), (3, "Cl3", "P3", 20, 800), (4, "Cl4", "P4", 15, 150),
(5, "Cl3", "P3", 10, 450), (6, "Cl2", "P2", 5, 800), (7, "Cl1", "P4", 22, 1200), (8, "Cl3", "P1", 15, 150), (9, "Cl1", "P1", 10, 500), (10, "Cl2", "P2", 5, 100);"""

# Populate Menu table
print("Inserting records into the tbl_orders table")
logger.info("Inserting records into the tbl_orders table")
first_cursor.execute(insert_into_tbl_order_query)
first_connection.commit()

Inserting records into the tbl_orders table


## Create a function to display results

In [21]:
# This function allows us to display the results in the mysql command line format

def get_max_number_from_columns(query: str):
    max_number_column = 0
    tables_in_query = []
    second_connection = pool.get_connection()
    second_cursor = second_connection.cursor()
    second_cursor.execute(f"USE {database_name}") 
    second_cursor.execute("SHOW TABLES;")
    tables = second_cursor.fetchall()
    for table in tables:
        tbl_name = table[0]
        if tbl_name in query:
            second_cursor.execute(f"SHOW COLUMNS FROM {tbl_name}")
            columns = second_cursor.fetchall()
            if "*" in query:
                for column in columns:
                    column_name = column[0]
                    max_query = f"SELECT MAX(LENGTH({column_name})) FROM {tbl_name}"
                    try:
                        second_cursor.execute(max_query)
                        result = second_cursor.fetchone()
                        if result[0] > max_number_column:
                            max_number_column = result[0]
                    except Error as err:
                        continue
            else:
                for column in columns:
                    column_name = column[0]
                    if column_name in query:
                        max_query = f"SELECT MAX(LENGTH({column_name})) FROM {tbl_name}"
                        try:
                            second_cursor.execute(max_query)
                            result = second_cursor.fetchone()
                            if result[0] > max_number_column:
                                max_number_column = result[0]
                        except Error as err:
                            continue
    second_cursor.close()
    second_connection.close()
    return max_number_column


def display_query_results(query: str): 
    logger.info(f"Executing the query: {query}")
    max_number_column = get_max_number_from_columns(query)
    first_cursor.execute(query)
    results = first_cursor.fetchall()
    table_column_names = first_cursor.column_names
    length = [len(x) for x in table_column_names]
    max_number_values =  max(length)
    max_number = max_number_column if max_number_column > max_number_values else max_number_values
    message = ""
    for _ in length:
        message = message + "+" + '-'*(max_number+2)
    message = message + "+"
    print(message)
    output = ""
    for column in table_column_names:
        output = output + f"| {column:^{max_number}} "
    output = output + "|"
    print(output)
    print(message)

    for result in results:
        display = ""
        for value in range(len(result)):
            text = result[value]
            if isinstance(text, datetime.date):
                text = str(text)
            if not text:
                text = "None"
            display = display + "|" + f"{text:^{max_number+2}}"
        print(display + "|")
    print(message)

In [22]:
## Show all the records in the table
select_query = "SELECT * FROM tbl_orders;"
display_query_results(select_query)

+-----------+-----------+-----------+-----------+-----------+
|  OrderID  | ClientID  | ProductID | Quantity  |   Cost    |
+-----------+-----------+-----------+-----------+-----------+
|     1     |    Cl1    |    P1     |    10     |  500.00   |
|     2     |    Cl2    |    P2     |     5     |  100.00   |
|     3     |    Cl3    |    P3     |    20     |  800.00   |
|     4     |    Cl4    |    P4     |    15     |  150.00   |
|     5     |    Cl3    |    P3     |    10     |  450.00   |
|     6     |    Cl2    |    P2     |     5     |  800.00   |
|     7     |    Cl1    |    P4     |    22     |  1200.00  |
|     8     |    Cl3    |    P1     |    15     |  150.00   |
|     9     |    Cl1    |    P1     |    10     |  500.00   |
|    10     |    Cl2    |    P2     |     5     |  100.00   |
+-----------+-----------+-----------+-----------+-----------+


# Filtering Task

## Task 1
Write a SQL statement to print all records of orders where the cost is $250 or less.

In [23]:
select_query = "SELECT * FROM tbl_orders WHERE Cost <= 250;"
display_query_results(select_query)

+-----------+-----------+-----------+-----------+-----------+
|  OrderID  | ClientID  | ProductID | Quantity  |   Cost    |
+-----------+-----------+-----------+-----------+-----------+
|     2     |    Cl2    |    P2     |     5     |  100.00   |
|     4     |    Cl4    |    P4     |    15     |  150.00   |
|     8     |    Cl3    |    P1     |    15     |  150.00   |
|    10     |    Cl2    |    P2     |     5     |  100.00   |
+-----------+-----------+-----------+-----------+-----------+


## Task 2
Write a SQL statement to print all records of orders where the cost is between \\$50 and \\$750.

In [24]:
select_query = "SELECT * FROM tbl_orders WHERE Cost BETWEEN 50 and 750;"
display_query_results(select_query)

+-----------+-----------+-----------+-----------+-----------+
|  OrderID  | ClientID  | ProductID | Quantity  |   Cost    |
+-----------+-----------+-----------+-----------+-----------+
|     1     |    Cl1    |    P1     |    10     |  500.00   |
|     2     |    Cl2    |    P2     |     5     |  100.00   |
|     4     |    Cl4    |    P4     |    15     |  150.00   |
|     5     |    Cl3    |    P3     |    10     |  450.00   |
|     8     |    Cl3    |    P1     |    15     |  150.00   |
|     9     |    Cl1    |    P1     |    10     |  500.00   |
|    10     |    Cl2    |    P2     |     5     |  100.00   |
+-----------+-----------+-----------+-----------+-----------+


## Task 3
Write a SQL statement to print all records of orders that have been placed by the client with the id of Cl3 and where the cost of the order is more than $100.

In [25]:
select_query = "SELECT * FROM tbl_orders WHERE ClientID = 'Cl3' AND Cost > 100;"
display_query_results(select_query)

+-----------+-----------+-----------+-----------+-----------+
|  OrderID  | ClientID  | ProductID | Quantity  |   Cost    |
+-----------+-----------+-----------+-----------+-----------+
|     3     |    Cl3    |    P3     |    20     |  800.00   |
|     5     |    Cl3    |    P3     |    10     |  450.00   |
|     8     |    Cl3    |    P1     |    15     |  150.00   |
+-----------+-----------+-----------+-----------+-----------+


## Task 4
Write a SQL statement to print all records of orders that have a product id of p1 or p2 and the order quantity is more than 2. 

In [26]:
select_query = "SELECT * FROM tbl_orders WHERE ProductID IN ('P1', 'P2') AND Quantity > 2;"
display_query_results(select_query)

+-----------+-----------+-----------+-----------+-----------+
|  OrderID  | ClientID  | ProductID | Quantity  |   Cost    |
+-----------+-----------+-----------+-----------+-----------+
|     1     |    Cl1    |    P1     |    10     |  500.00   |
|     2     |    Cl2    |    P2     |     5     |  100.00   |
|     6     |    Cl2    |    P2     |     5     |  800.00   |
|     8     |    Cl3    |    P1     |    15     |  150.00   |
|     9     |    Cl1    |    P1     |    10     |  500.00   |
|    10     |    Cl2    |    P2     |     5     |  100.00   |
+-----------+-----------+-----------+-----------+-----------+


# Joins

In [27]:
## Create additional tables
create_table_customers_query = """CREATE TABLE IF NOT EXISTS tbl_customers(CustomerID INT NOT NULL PRIMARY KEY, FullName VARCHAR(100) NOT NULL, PhoneNumber INT NOT NULL UNIQUE);"""
create_table_bookings_query = """CREATE TABLE IF NOT EXISTS tbl_bookings (BookingID INT NOT NULL PRIMARY KEY,  BookingDate DATE NOT NULL, TableNumber INT NOT NULL, NumberOfGuests INT NOT NULL CHECK (NumberOfGuests <= 8), 
CustomerID INT NOT NULL, FOREIGN KEY (CustomerID) REFERENCES tbl_customers (CustomerID) ON DELETE CASCADE ON UPDATE CASCADE); """

first_cursor.execute(create_table_customers_query)
print("Table tbl_customers successfully created")
logger.info("Table tbl_customers successfully created")
first_cursor.execute(create_table_bookings_query)
print("Table tbl_bookings successfully created")
logger.info("Table tbl_bookings successfully created")

Table tbl_customers successfully created
Table tbl_bookings successfully created


In [28]:
# Check to that the two additional tables were created in the database
first_cursor.execute("SHOW TABLES;")
results = first_cursor.fetchall()
assert len(results) == 3
for table in results:
    print(table[0]) 

tbl_bookings
tbl_customers
tbl_orders


In [29]:
## Populate the tables with Data
insert_into_tbl_customers = """INSERT INTO tbl_customers(CustomerID, FullName, PhoneNumber) VALUES (1, "Vanessa McCarthy", 0757536378), (2, "Marcos Romero", 0757536379), (3, "Hiroki Yamane", 0757536376), (4, "Anna Iversen", 0757536375), 
(5, "Diana Pinto", 0757536374);"""
logger.info("Inserting records into the tbl_customers table")
first_cursor.execute(insert_into_tbl_customers)
first_connection.commit()

insert_into_tbl_bookings = """INSERT INTO tbl_bookings (BookingID, BookingDate, TableNumber, NumberOfGuests, CustomerID) VALUES (10, '2021-11-11', 7, 5, 1), (11, '2021-11-10', 5, 2, 2), (12, '2021-11-10', 3, 2, 4);"""
logger.info("Inserting records into the tbl_bookings table")
first_cursor.execute(insert_into_tbl_bookings)
first_connection.commit()

## Task 1
Write an INNER JOIN SQL statement to combine the full name and the phone number of each customer from the Customers table with the related booking date and 'number of guests' from the Bookings table. 

In [30]:
inner_join_query = """SELECT FullName, PhoneNumber, BookingDate, NumberOfGuests FROM tbl_customers
INNER JOIN tbl_bookings ON tbl_customers.CustomerID = tbl_bookings.CustomerID;"""
display_query_results(inner_join_query)

+------------------+------------------+------------------+------------------+
|     FullName     |   PhoneNumber    |   BookingDate    |  NumberOfGuests  |
+------------------+------------------+------------------+------------------+
| Vanessa McCarthy |    757536378     |    2021-11-11    |        5         |
|  Marcos Romero   |    757536379     |    2021-11-10    |        2         |
|   Anna Iversen   |    757536375     |    2021-11-10    |        2         |
+------------------+------------------+------------------+------------------+


## Task 2
Write a LEFT JOIN SQL statement to view the customer id from Customers table and the related booking id from the Bookings table.

In [31]:
left_join_query = """SELECT tbl_customers.CustomerID, BookingID FROM tbl_customers
LEFT JOIN tbl_bookings ON tbl_customers.CustomerID = tbl_bookings.CustomerID;"""
display_query_results(left_join_query)

+------------+------------+
| CustomerID | BookingID  |
+------------+------------+
|     5      |    None    |
|     4      |     12     |
|     3      |    None    |
|     1      |     10     |
|     2      |     11     |
+------------+------------+


# Group By

In [32]:
create_new_order_table = """CREATE TABLE orders_tbl(OrderID INT, Department VARCHAR(100), OrderDate DATE, OrderQty INT, OrderTotal INT, PRIMARY KEY(OrderID));"""
first_cursor.execute(create_new_order_table)
print("Table orders_tbl successfully created")
logger.info("Table orders_tbl successfully created")

Table orders_tbl successfully created


In [33]:
insert_into_new_table = """INSERT INTO orders_tbl VALUES(1,'Lawn Care','2022-05-05',12,500),(2,'Decking','2022-05-22',150,1450),(3,'Compost and Stones','2022-05-27',20,780),(4,'Trees and Shrubs','2022-06-01',15,400),(5,'Garden Decor','2022-06-10',2,1250),
(6,'Lawn Care','2022-06-10',12,500),(7,'Decking','2022-06-25',150,1450),(8,'Compost and Stones','2022-05-29',20,780),(9,'Trees and Shrubs','2022-06-10',15,400),(10,'Garden Decor','2022-06-10',2,1250),(11,'Lawn Care','2022-06-25',10,400), 
(12,'Decking','2022-06-25',100,1400),(13,'Compost and Stones','2022-05-30',15,700),(14,'Trees and Shrubs','2022-06-15',10,300),(15,'Garden Decor','2022-06-11',2,1250),(16,'Lawn Care','2022-06-10',12,500),
(17,'Decking','2022-06-25',150,1450),(18,'Trees and Shrubs','2022-06-10',15,400),(19,'Lawn Care','2022-06-10',12,500),(20,'Decking','2022-06-25',150,1450),(21,'Decking','2022-06-25',150,1450);"""


logger.info("Inserting records into the orders_tbl table")
first_cursor.execute(insert_into_new_table)
first_connection.commit()

In [34]:
select_query = "SELECT * FROM orders_tbl"
display_query_results(select_query)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|      OrderID       |     Department     |     OrderDate      |      OrderQty      |     OrderTotal     |
+--------------------+--------------------+--------------------+--------------------+--------------------+
|         1          |     Lawn Care      |     2022-05-05     |         12         |        500         |
|         2          |      Decking       |     2022-05-22     |        150         |        1450        |
|         3          | Compost and Stones |     2022-05-27     |         20         |        780         |
|         4          |  Trees and Shrubs  |     2022-06-01     |         15         |        400         |
|         5          |    Garden Decor    |     2022-06-10     |         2          |        1250        |
|         6          |     Lawn Care      |     2022-06-10     |         12         |        500         |
|         7          |      Decking  

## Task 1
Write a SQL SELECT statement to group all records that have the same order date.

In [35]:
select_query = "SELECT OrderDate FROM orders_tbl GROUP BY OrderDate"
display_query_results(select_query)

+------------+
| OrderDate  |
+------------+
| 2022-05-05 |
| 2022-05-22 |
| 2022-05-27 |
| 2022-06-01 |
| 2022-06-10 |
| 2022-06-25 |
| 2022-05-29 |
| 2022-05-30 |
| 2022-06-15 |
| 2022-06-11 |
+------------+


## Task 2
Write a SQL SELECT statement to retrieve the number of orders placed on the same day.

In [36]:
select_query = "SELECT OrderDate, Count(OrderQty) AS OrderQuantity FROM orders_tbl GROUP BY OrderDate"
display_query_results(select_query)

+---------------+---------------+
|   OrderDate   | OrderQuantity |
+---------------+---------------+
|  2022-05-05   |       1       |
|  2022-05-22   |       1       |
|  2022-05-27   |       1       |
|  2022-06-01   |       1       |
|  2022-06-10   |       7       |
|  2022-06-25   |       6       |
|  2022-05-29   |       1       |
|  2022-05-30   |       1       |
|  2022-06-15   |       1       |
|  2022-06-11   |       1       |
+---------------+---------------+


## Task 3
Write a SQL SELECT statement to retrieve the total order quantities placed by each department.

In [37]:
select_query = "SELECT Department, Sum(OrderQty) AS OrderQuantity FROM orders_tbl GROUP BY Department"
display_query_results(select_query)

+--------------------+--------------------+
|     Department     |   OrderQuantity    |
+--------------------+--------------------+
|     Lawn Care      |         58         |
|      Decking       |        850         |
| Compost and Stones |         55         |
|  Trees and Shrubs  |         55         |
|    Garden Decor    |         6          |
+--------------------+--------------------+


## Task 4
Write a SQL SELECT statement to retrieve the number of orders placed on the same day between the following dates: 1st June 2022 and 30th June 2022.

In [38]:
select_query = "SELECT OrderDate, Count(OrderQty) AS OrderQuantity FROM orders_tbl WHERE OrderDate BETWEEN '2022-06-01' AND '2022-06-30' GROUP BY OrderDate"
display_query_results(select_query)

+---------------+---------------+
|   OrderDate   | OrderQuantity |
+---------------+---------------+
|  2022-06-01   |       1       |
|  2022-06-10   |       7       |
|  2022-06-25   |       6       |
|  2022-06-15   |       1       |
|  2022-06-11   |       1       |
+---------------+---------------+


# Replace Into
How the REPLACE INTO statement works
The REPLACE statement checks whether the intended data record's unique key value already exists in the table before inserting it as a new record or updating it. 

The REPLACE INTO statement attempts to insert a new record or modify an existing record. In both cases, it checks whether the unique key of the proposed record already exists in the table. Suppose a value of NO or FALSE is returned. In that case, the REPLACE statement inserts the record similar to the INSERT INTO statement.

Suppose the key value already exists in the table (in other words, a duplicate key). In that case, the REPLACE statement deletes the existing record of data and replaces it with a new record of data. This happens regardless of whether you use the first or the second REPLACE statement syntax.

```sql
REPLACE INTO table_name (column1name, column2name, ...) 
VALUES (value1, value2, ...);
```


```sql
REPLACE INTO table_name SET column1name = value, column2name = value, ... ;
```

In [39]:
select_query = "SELECT * FROM tbl_orders"
display_query_results(select_query)

+-----------+-----------+-----------+-----------+-----------+
|  OrderID  | ClientID  | ProductID | Quantity  |   Cost    |
+-----------+-----------+-----------+-----------+-----------+
|     1     |    Cl1    |    P1     |    10     |  500.00   |
|     2     |    Cl2    |    P2     |     5     |  100.00   |
|     3     |    Cl3    |    P3     |    20     |  800.00   |
|     4     |    Cl4    |    P4     |    15     |  150.00   |
|     5     |    Cl3    |    P3     |    10     |  450.00   |
|     6     |    Cl2    |    P2     |     5     |  800.00   |
|     7     |    Cl1    |    P4     |    22     |  1200.00  |
|     8     |    Cl3    |    P1     |    15     |  150.00   |
|     9     |    Cl1    |    P1     |    10     |  500.00   |
|    10     |    Cl2    |    P2     |     5     |  100.00   |
+-----------+-----------+-----------+-----------+-----------+


## Task 1
Write a SQL REPLACE statement that inserts two new orders with the following details:

Order 11 data:
- OrderID = 11, 
- ClientID = "Cl2", 
- ProductID = "P3", 
- Quantity = 50, 
- Cost = 5000

Order 12 data:
- OrderID = 12, 
- ClientID = "Cl2", 
- ProductID = "P2", 
- Quantity = 25, 
- Cost = 1000

In [40]:
replace_query = """REPLACE INTO tbl_orders (OrderID, ClientID, ProductID, Quantity, Cost) 
VALUES (11, 'Cl2', 'P3', 50, 5000), (12, 'Cl2', 'P2', 25, 1000);"""

first_cursor.execute(replace_query)
first_connection.commit()

In [41]:
select_query = "SELECT * FROM tbl_orders"
display_query_results(select_query)

+-----------+-----------+-----------+-----------+-----------+
|  OrderID  | ClientID  | ProductID | Quantity  |   Cost    |
+-----------+-----------+-----------+-----------+-----------+
|     1     |    Cl1    |    P1     |    10     |  500.00   |
|     2     |    Cl2    |    P2     |     5     |  100.00   |
|     3     |    Cl3    |    P3     |    20     |  800.00   |
|     4     |    Cl4    |    P4     |    15     |  150.00   |
|     5     |    Cl3    |    P3     |    10     |  450.00   |
|     6     |    Cl2    |    P2     |     5     |  800.00   |
|     7     |    Cl1    |    P4     |    22     |  1200.00  |
|     8     |    Cl3    |    P1     |    15     |  150.00   |
|     9     |    Cl1    |    P1     |    10     |  500.00   |
|    10     |    Cl2    |    P2     |     5     |  100.00   |
|    11     |    Cl2    |    P3     |    50     |  5000.00  |
|    12     |    Cl2    |    P2     |    25     |  1000.00  |
+-----------+-----------+-----------+-----------+-----------+


## Task 2
It was noticed that the cost of order number 11 is \\\$5000. This is a mistake. The order must cost \\$500. You must help them to change it to \\$500 by writing a relevant REPLACE statement. 

In [42]:
replace_query_set = """REPLACE INTO tbl_orders SET OrderID = 11, ClientID = 'Cl2', ProductID = 'P3', Quantity = 50, Cost = 500;"""
first_cursor.execute(replace_query_set)
first_connection.commit()

In [43]:
select_query = "SELECT * FROM tbl_orders"
display_query_results(select_query)

+-----------+-----------+-----------+-----------+-----------+
|  OrderID  | ClientID  | ProductID | Quantity  |   Cost    |
+-----------+-----------+-----------+-----------+-----------+
|     1     |    Cl1    |    P1     |    10     |  500.00   |
|     2     |    Cl2    |    P2     |     5     |  100.00   |
|     3     |    Cl3    |    P3     |    20     |  800.00   |
|     4     |    Cl4    |    P4     |    15     |  150.00   |
|     5     |    Cl3    |    P3     |    10     |  450.00   |
|     6     |    Cl2    |    P2     |     5     |  800.00   |
|     7     |    Cl1    |    P4     |    22     |  1200.00  |
|     8     |    Cl3    |    P1     |    15     |  150.00   |
|     9     |    Cl1    |    P1     |    10     |  500.00   |
|    10     |    Cl2    |    P2     |     5     |  100.00   |
|    11     |    Cl2    |    P3     |    50     |  500.00   |
|    12     |    Cl2    |    P2     |    25     |  1000.00  |
+-----------+-----------+-----------+-----------+-----------+


In [44]:
first_cursor.close()
first_connection.close()